In [2]:
# CNN Model Training using PyTorch - Basic Model (PHASE 3)

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

# Loading .npy path files
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val   = np.load("X_val.npy")
y_val   = np.load("y_val.npy")

# Now creating a Custom Dataset
class PatchDataset(Dataset):
    def __init__(self, X, y, transform = None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx].astype(np.float32)
        label = self.y[idx]
        image = torch.tensor(image).permute(2, 0, 1)  # From HWC to CHW because of PyTorch
        label = torch.tensor(label).float()
        return image, label

# Loading Data
train_dataset = PatchDataset(X_train, y_train)
val_dataset = PatchDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size= 32)

# Basic CNN Model
class TamperCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 64x64
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 32x32
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 16x16
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, 1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).squeeze()

model = TamperCNN()

# Training Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
train_loss_hist, val_loss_hist = [], []
for epoch in range(15):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            preds = model(X_batch)
            val_loss += loss_fn(preds, y_batch).item()

    train_loss_hist.append(total_loss / len(train_loader))
    val_loss_hist.append(val_loss / len(val_loader))

    print(f"Epoch {epoch+1} | Train Loss: {train_loss_hist[-1]:.4f} | Val Loss: {val_loss_hist[-1]:.4f}")

# Saving the Basic Model
torch.save(model.state_dict(), "tampered_patch_cnn.pth")
print("Model saved as tampered_patch_cnn.pth")

Epoch 1 | Train Loss: 0.6871 | Val Loss: 0.6742
Epoch 2 | Train Loss: 0.6522 | Val Loss: 0.6416
Epoch 3 | Train Loss: 0.6212 | Val Loss: 0.6214
Epoch 4 | Train Loss: 0.5965 | Val Loss: 0.6057
Epoch 5 | Train Loss: 0.5762 | Val Loss: 0.5824
Epoch 6 | Train Loss: 0.5485 | Val Loss: 0.6036
Epoch 7 | Train Loss: 0.5471 | Val Loss: 0.5590
Epoch 8 | Train Loss: 0.5245 | Val Loss: 0.5604
Epoch 9 | Train Loss: 0.5036 | Val Loss: 0.5548
Epoch 10 | Train Loss: 0.5065 | Val Loss: 0.5495
Epoch 11 | Train Loss: 0.4811 | Val Loss: 0.5510
Epoch 12 | Train Loss: 0.4778 | Val Loss: 0.5458
Epoch 13 | Train Loss: 0.4665 | Val Loss: 0.5391
Epoch 14 | Train Loss: 0.4668 | Val Loss: 0.5503
Epoch 15 | Train Loss: 0.4615 | Val Loss: 0.5426
Model saved as tampered_patch_cnn.pth
